In [27]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_d4.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_b5.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_ea.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_a3.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_9a.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_98.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_ee.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_ca.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_88.jpg
/kaggle/input/ident

In [28]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import os

In [29]:
num_classes = 8

In [30]:
dataset_dir = '/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset'

In [31]:
#Create an instance of ImageDataGenerator for data augmentation and preprocessing
datagen = ImageDataGenerator(rescale=1./255,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             validation_split=0.2)  # 80% training, 20% validation

In [32]:
# Create a generator for validation data
val_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    subset='validation')  # Use the subset parameter for validation data

Found 174 images belonging to 8 classes.


In [33]:
# Create a generator for training data
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    subset='training')

Found 711 images belonging to 8 classes.


In [34]:
# Create a generator for test data
test_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    shuffle=False)  # Disable shuffling for consistent results

Found 885 images belonging to 8 classes.


In [35]:
dataset = datagen.flow_from_directory(dataset_dir,
                                             target_size=(299, 299),
                                             batch_size=32,
                                             class_mode='categorical',
                                             subset='training')

Found 711 images belonging to 8 classes.


In [36]:
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(dataset.num_classes, activation='softmax')(x)

In [37]:
model = Model(inputs=base_model.input, outputs=predictions)

In [38]:
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define a callback to calculate training accuracy
class TrainingAccuracyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        train_loss, train_accuracy = self.model.evaluate(dataset, steps=dataset.samples // batch_size)
        print('Training loss:', train_loss)
        print('Training accuracy:', train_accuracy)

In [39]:
# Train the model
batch_size=32
num_train_samples = dataset.samples
num_val_samples = val_generator.samples
epochs = 20

model.fit(dataset,
          steps_per_epoch=num_train_samples // 32,
          validation_data=val_generator,
          validation_steps=num_val_samples // 32,
          epochs=epochs,
          callbacks=[TrainingAccuracyCallback()])

Epoch 1/20
22/22 [==============================] - 41s 2s/step - loss: 0.8433 - accuracy: 0.6989
Training loss: 0.8433027267456055
Training accuracy: 0.6988636255264282
22/22 [==============================] - 138s 4s/step - loss: 1.5143 - accuracy: 0.4919 - val_loss: 1.1513 - val_accuracy: 0.5875
Epoch 2/20
22/22 [==============================] - 41s 2s/step - loss: 0.5560 - accuracy: 0.7855
Training loss: 0.5560163855552673
Training accuracy: 0.7855113744735718
22/22 [==============================] - 102s 5s/step - loss: 0.7297 - accuracy: 0.7496 - val_loss: 0.9423 - val_accuracy: 0.6313
Epoch 3/20
22/22 [==============================] - 45s 2s/step - loss: 0.4592 - accuracy: 0.8381
Training loss: 0.4592253565788269
Training accuracy: 0.8380681872367859
22/22 [==============================] - 104s 5s/step - loss: 0.5170 - accuracy: 0.8130 - val_loss: 0.8852 - val_accuracy: 0.7063
Epoch 4/20
22/22 [==============================] - 45s 2s/step - loss: 0.3412 - accuracy: 0.8949
Tr

In [40]:
# model.save('mobilenetv2_model.h5')

In [41]:
# Evaluate the model on training data
train_loss, train_acc = model.evaluate(train_generator, steps=len(train_generator))
print(f'Training accuracy: {train_acc}')
print(f'Training loss: {train_loss}')

23/23 [==============================] - 46s 2s/step - loss: 0.0633 - accuracy: 0.9930
Training accuracy: 0.9929676651954651
Training loss: 0.06327134370803833


In [42]:
# Evaluate the model on validation data
val_loss, val_acc = model.evaluate(val_generator, steps=len(val_generator))
print(f'Validation accuracy: {val_acc}')
print(f'Validation loss: {val_loss}')

6/6 [==============================] - 12s 2s/step - loss: 1.0153 - accuracy: 0.7069
Validation accuracy: 0.7068965435028076
Validation loss: 1.0152770280838013


In [43]:
#Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
print(f'Test accuracy: {test_acc}')
print(f'Test loss: {test_loss}')

28/28 [==============================] - 59s 2s/step - loss: 0.2301 - accuracy: 0.9333
Test accuracy: 0.9333333373069763
Test loss: 0.23009368777275085
